In [ ]:
# !pip install geemap

<h1><center>Monitoring Grassland Health in Mongolia </center></h1>

**Steps to create soum level change detection map:**

1. Choose Province
2. Choose Soum
3. Choose satellite derived index (more about these indices: https://www.usgs.gov/landsat-missions/landsat-surface-reflectance-derived-spectral-indices)
4. Select Year
5. Check `Only Pasture` (This will display areas that are deemed to be only grazing lands)
6. Check `Apply fmask(remove cloud, shadow, snow)` (This will use only images without any cloud, shadow)

**How to intrepret the map?**

The algorithm display vegetation change map for a given year for a given soum. The color maps can be intrepreted in relative to the mean of 8 years between 2014-2021. In other words, a color map indicates how much given year's vegetation deviate from from the mean value between 2014-2021. The three colors - green, yellow, red - represents the standard deviation distance. Green displays values where distance from mean is positive, yellow neatrul, and red negative. Thus, the intensity of the color indicates the larger distance from the mean.    

**Web Apps:** https://nogoonzun.herokuapp.com/

**Contact:** Khusel Avirmed (ta346@cornell.edu)

In [ ]:
# Check geemap installation
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package is not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [5]:
# Import libraries
import os
import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
import ee

In [6]:
# key_dir = './greensummer-web-265dfc893fdd.json'
service_account = 'greensummer@greensummer-web.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'greensummer-web-265dfc893fdd.json')
ee.Initialize(credentials)


AttributeError: module 'collections' has no attribute 'Callable'

In [7]:
# try:
#         ee.Initialize()
# except Exception as e:
#         ee.Authenticate()
#         ee.Initialize()

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
#import altair as alt
import numpy as np

In [ ]:
# import local sources
import sys

current_dir = os.getcwd()
script_path = os.path.join(current_dir, 'script')
sys.path.append(script_path)


import gee_script.landsat_functions
import gee_script.indices
import gee_script.mask
import gee_script.utils

In [ ]:
# Create an interactive map
Map = geemap.Map()
# Add Earth Engine data
fc = ee.FeatureCollection('users/ta346/mng-bounds/soum_aimag')

In [ ]:
# Map.add_basemap('ROAD')
Map.addLayer(ee.Image().paint(fc, 0, 2), {'palette': 'black'}, 'Soum Administrative Units')
Map.centerObject(fc, 5)
Map

In [ ]:
# bring soum province names as list of dictionaries
import json
with open("soum_province_names.json", "r") as read_file:
    newdict = json.load(read_file)

In [ ]:
# Designe interactive widgets
style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='topleft')
Map.add_control(output_control)

# admin1_widget = widgets.Text(
#     description='State:',
#     value='Tennessee',
#     width=200,
#     style=style
# )

dc = 'Arkhangai'
province = widgets.Dropdown(
    options = list(newdict),
    description = 'Province:',
    disabled = False,
)

soum = widgets.Dropdown(
    options=newdict[dc],
    value = "Chuluut",
    description = 'Soum:',
    disabled = False,
)

def on_value_change(change):
    dc = change.new
    soum.options = newdict[dc]

province.observe(on_value_change, 'value')

nd_options = ['Normalized Difference Vegetation Index (NDVI)', 
              'Normalized Difference Water Index (NDWI)',
              'Modified Soil Adjusted Vegetation Index (MSAVI)',
              'Enhanced Vegetation Index (EVI)',
              'Near-Infrared Reflectence Vegetation (NIRv)']

nd_indices = widgets.Dropdown(options=nd_options, 
                              value='Normalized Difference Vegetation Index (NDVI)', 
                              description='Satellite Derived Index:', 
                              style=style)

year_widget = widgets.IntSlider(min=2014, max=2022, value=2014, description='Selected year:', width=400, style=style)

pasture_widget = widgets.Checkbox(
    value=True,
    description='Only pasture',
    style=style
)

fmask_widget = widgets.Checkbox(
    value=True,
    description='Apply fmask(remove cloud, shadow, snow)',
    style=style
)

submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

full_widget = widgets.VBox([
    widgets.VBox([province, soum, nd_indices, year_widget, pasture_widget, fmask_widget]),
    submit
])

full_widget

In [ ]:
# create date range from today to past 20 years
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-20, 'years'), today)

In [ ]:
layer_name = []
soum_layer_name = []
# Click event handler
def submit_clicked(b):

    if layer_name:
        try:
            Map.remove_ee_layer(layer_name[0])
        except Exception as e:
            print(e)
    else:
        Map.remove_ee_layer(None)
    
    if soum_layer_name:
        try:
            Map.remove_ee_layer(soum_layer_name[0])
        except Exception as e:
            print(e)
    else:
        Map.remove_ee_layer(None)

    with output_widget:
        
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}
        
        
        try:
            province_id = province.value 
            soum_id = soum.value
            nd_indices_id = nd_indices.value
            selected_year = year_widget.value 
            pasture_yes = pasture_widget.value 
            apply_fmask = fmask_widget.value
            
            # select aimag and soum
            soum_aimag = fc.filter(ee.Filter.And(ee.Filter.eq("aimag_eng", province_id), ee.Filter.eq("soum_eng", soum_id)))
            prov_eng_name = province_id + " " + soum_id
            soum_layer_name.insert(0, prov_eng_name)
            geom = soum_aimag.geometry()
            Map.addLayer(ee.Image().paint(soum_aimag, 0, 2), {'palette': 'red'}, soum_layer_name[0])
#             Map.addLayer(ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name)
            
            # cloud free landsat collection
            landsat_collection = (main.get_landsat_collection(dateIni='2014-01-01', # initial date
                                                                    dateEnd='2022-12-31', # end date
                                                                    box=geom, # area of interest
                                                                    cloud_cover_perc = 20,
                                                                    sensor=["LC08", "LE07", "LT05"], # LC08, LE07, LT05, search for all available sensors
                                                                    harmonization=True)) # ETM and ETM+ to OLI
            summer_composite = (main.make_composite(landsat_collection, 6, 8, geom)).select('ndvi')
            
            # Compute vegetation indices on cloud free landsat collection
            if nd_indices_id == 'Normalized Difference Vegetation Index (NDVI)':
                summer_composite = (main.make_composite(landsat_collection, 6, 8, geom)).select('ndvi')
            elif nd_indices_id == 'Normalized Difference Water Index (NDWI)':
                summer_composite = (main.make_composite(landsat_collection, 6, 8, geom)).select('ndwi')
            elif nd_indices_id == 'Modified Soil Adjusted Vegetation Index (MSAVI)':
                summer_composite = (main.make_composite(landsat_collection, 6, 8, geom)).select('msavi')
            elif nd_indices_id == 'Enhanced Vegetation Index (EVI)':
                summer_composite = (main.make_composite(landsat_collection, 6, 8, geom)).select('evi')
            elif nd_indices_id == 'Near-Infrared Reflectence Vegetation (NIRv)':
                summer_composite = (main.make_composite(landsat_collection, 6, 8, geom)).select('nirv')
            
            if pasture_yes:
                summer_pasture_mask = ee.Image("users/ta346/pasture_delineation/pas_raster_new")
                
                summer_composite = summer_composite.map(mask.image_mask(summer_pasture_mask, [3]))
                
            dateIni = ee.Date.fromYMD(selected_year, 1, 1)
            dateEnd = ee.Date.fromYMD(selected_year, 12, 31)
            
            selected_image = summer_composite.filterDate(dateIni, dateEnd).first()
            

            yMean = summer_composite.mean()
            stdImg = summer_composite.reduce(ee.Reducer.stdDev())
            
            Anomaly = selected_image.subtract(yMean).divide(stdImg).clip(geom)


            colorizedVis = {
                'min': 0.0,
                'max': 1.0,
                'palette': [
                    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
                    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
                    '012E01', '011D01', '011301'
                ]}
            
            anomParams = {'min': -3, 'max':3, 'palette': ['red', 'yellow', 'green']}
            anom_layer_name = str(selected_year) + ' ' + str(nd_indices_id)
            layer_name.insert(0, anom_layer_name)

            Map.addLayer(Anomaly, anomParams, anom_layer_name)
            Map.center_object(soum_aimag, 9)
#             Map.remove_ee_layer(anom_layer_name)
            
            Map.default_style = {'cursor': 'default'}
            
        except Exception as e:
            print(e)
            print('An error occurred during computation.')
            
submit.on_click(submit_clicked)

In [ ]:
date = "2012-11-01"
ee_date = ee.Date(date)
int(ee_date.format('YYYY').getInfo())

## Drought

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

In [ ]:
def submit_clicked_chart(a):
    with out:
        out.clear_output()
        try:
            pdsi = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate(date_range).select('pdsi')
            # aoi = ee.FeatureCollection('EPA/Ecoregions/2013/L3').filter(
            #     ee.Filter.eq('na_l3name', 'Sierra Nevada')).geometry()
            # pdsi = ee.ImageCollection('GRIDMET/DROUGHT').select('pdsi')

            # define area of interest
            aoi = fc.filter(ee.Filter.And(ee.Filter.eq("aimag_eng", province.value), ee.Filter.eq("soum_eng", soum.value))).geometry()
            # aoi = ee.FeatureCollection(fc).filter(ee.Filter.eq('asid', 8443)).geometry()
            # aoi_subset_collection = ee.FeatureCollection(fc).filter(ee.Filter.inList('asid', ee.List([4173, 4176])))

            # Reduce data

            # 1. Create a region reduction function.
            # 2. Map the function over the `pdsi` image collection to reduce each image.
            # 3. Filter out any resulting features that have null computed values (occurs when all pixels in an AOI are masked).
            reduce_pdsi = utils.create_reduce_region_function(
                geometry=aoi, reducer=ee.Reducer.mean(), scale=5000, crs='EPSG:4326')

            pdsi_stat_fc = ee.FeatureCollection(pdsi.map(reduce_pdsi)).filter(
                ee.Filter.notNull(pdsi.first().bandNames()))

            # Server to client transfer
            pdsi_dict = utils.fc_to_dict(pdsi_stat_fc).getInfo()

            # convert to Panda dataframe
            pdsi_df = pd.DataFrame(pdsi_dict)

            # add date variables to Dataframe function to the Dataframe
            pdsi_df = utils.add_date_info(pdsi_df)

            # more cleaning
            pdsi_df = pdsi_df.rename(columns={
                'pdsi': 'PDSI'
            }).drop(columns=['millis', 'system:index'])

            pdsi_df['PDSI'] = pdsi_df['PDSI'] / 100

            # calendar map
            a = alt.Chart(pdsi_df).mark_rect().encode(
                x='Year:O',
                y='Month:O',
                color=alt.Color(
                    'mean(PDSI):Q', scale=alt.Scale(scheme='redblue', domain=(-10, 10))),
                tooltip=[
                    alt.Tooltip('Year:O', title='Year'),
                    alt.Tooltip('Month:O', title='Month'),
                    alt.Tooltip('mean(PDSI):Q', title='PDSI')
                ]).properties(width=600, height=300)

            a.display()
            
            b = alt.Chart(pdsi_df).mark_bar(size=3).encode(
                x='Timestamp:T',
                y='PDSI:Q',
                color=alt.Color(
                    'PDSI:Q', scale=alt.Scale(scheme='redblue', domain=(-10, 10))),
                tooltip=[
                    alt.Tooltip('Timestamp:T', title='Date'),
                    alt.Tooltip('PDSI:Q', title='PDSI')
                ]).properties(width=600, height=300)
            
            b.display()
            
        except Exception as e:
            print(e)
            print('An error occurred during computation.')

submit.on_click(submit_clicked_chart)

## Vegetation productivity

In [ ]:
out_chart_2 = widgets.Output(layout = {'border': '1px solid black'})
display(out_chart_2)

In [ ]:
def submit_click_chart2(a):
    with out_chart_2:
        out_chart_2.clear_output()
        try:
            ndvi = ee.ImageCollection('MODIS/006/MOD13A2').filterDate(date_range).select('NDVI')
            
            # define area of interest as it changes from widget boxes
            aoi = fc.filter(ee.Filter.And(ee.Filter.eq("aimag_eng", province.value), ee.Filter.eq("soum_eng", soum.value))).geometry()
            
            # prepare for reducing the available images
            reduce_ndvi = utils.create_reduce_region_function(
                geometry = aoi, reducer = ee.Reducer.mean(), scale = 1000, crs = 'EPSG: 4326')
            
            # apply reduction function to produce FeatureCollection
            ndvi_stat_fc = ee.FeatureCollection(ndvi.map(reduce_ndvi)).filter(
                ee.Filter.notNull(ndvi.first().bandNames()))
            
            # Server to client converstion
            ndvi_dict = utils.fc_to_dict(ndvi_stat_fc).getInfo()
            
            # read in Panda dataframe
            ndvi_df = pd.DataFrame(ndvi_dict)
            
            # some cleaning and add dates
            ndvi_df['NDVI'] = ndvi_df['NDVI'] / 10000
            ndvi_df = utils.add_date_info(ndvi_df)
            
            # graphing
            highlight = alt.selection(
                type = 'single', on = 'mouseover', fields = ['Year'], nearest = True)
            
            base = alt.Chart(ndvi_df).encode(
                x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 350], clamp=True)),
                y=alt.Y('NDVI:Q', scale=alt.Scale(domain=[0.00, max(ndvi_df['NDVI'])])),
                color=alt.Color('Year:O', scale=alt.Scale(scheme='magma')))

            points = base.mark_circle().encode(
                opacity=alt.value(0),
                tooltip=[
                    alt.Tooltip('Year:O', title='Year'),
                    alt.Tooltip('DOY:Q', title='DOY'),
                    alt.Tooltip('NDVI:Q', title='NDVI')
                ]).add_selection(highlight)

            lines = base.mark_line().encode(
                size=alt.condition(~highlight, alt.value(1), alt.value(3)))
            
            a = (points + lines).properties(width = 600, height = 350).interactive()
            
            a.display()
            
            base2 = alt.Chart(ndvi_df).encode(
                x=alt.X('DOY:Q', scale=alt.Scale(domain=(0, 350))))

            line2 = base2.mark_line().encode(
                y=alt.Y('median(NDVI):Q', scale=alt.Scale(domain=(0.00, max(ndvi_df['NDVI'])))))

            band2 = base2.mark_errorband(extent='iqr').encode(
                y='NDVI:Q')

            b = (line2 + band2).properties(width=600, height=300).interactive()
            
            b.display()
        
        except Exception as e:
            print(e)
            print('An error occurred during computation.')

submit.on_click(submit_click_chart2)

## Past and future climate

In [ ]:
out_chart_3 = widgets.Output(layout = {'border': '1px solid black'})
display(out_chart_3)

In [ ]:
def submit_click_chart3(a):
    with out_chart_3:
        out_chart_3.clear_output()
        try:
            #### Future climate
            startDate = '2020-01-01'
            endDate = '2025-12-31'

            monthDifference = ee.Date(startDate).advance(1, 'month').millis().subtract(ee.Date(startDate).millis())
            listMap = ee.List.sequence(ee.Date(startDate).millis(), ee.Date(endDate).millis(), monthDifference)

            def monthly_temp_composite(dateMillis):
                date = ee.Date(dateMillis)
                nasa_future = (ee.ImageCollection('NASA/NEX-GDDP')
                              .select(['tasmax', 'tasmin', 'pr'])
                              .filter(ee.Filter.eq('scenario', 'rcp85'))
                              .filterDate(date, date.advance(1, 'month'))
                              .mean()
                              .set('system:index', date.format())
                              .set('system:time_start', date.millis()) )                  
                return nasa_future

            nasa_future_climate = ee.ImageCollection.fromImages(listMap.map(monthly_temp_composite))
            # nasa_future_month_pr = ee.ImageCollection.fromImages(listMap.map(monthly_pr_composite))

            # nasa_future_climate = nasa_future_month_temp.combine(nasa_future_month_pr)


            def calc_mean_temp(img):
                return (img.select('tasmax') \
                        .add(img.select('tasmin')) \
                        .divide(ee.Image.constant(2.0)) \
                        .addBands(img.select('pr')) \
                        .rename(['Temp-mean', 'Precip-rate']) \
                        .copyProperties(img, img.propertyNames()))

            nasa_future_month = nasa_future_climate.map(calc_mean_temp)
            
            # define area of interest as it changes from widget boxes
            aoi = fc.filter(ee.Filter.And(ee.Filter.eq("aimag_eng", province.value), ee.Filter.eq("soum_eng", soum.value))).geometry()
            
            reduce_future_climate = utils.create_reduce_region_function(
                geometry=aoi, reducer=ee.Reducer.mean(), scale=5000, crs='EPSG:4326')

            future_climate_stat_fc = ee.FeatureCollection(nasa_future_month.map(reduce_future_climate)).filter(
                ee.Filter.notNull(nasa_future_month.first().bandNames()))

            dcp_dict = utils.fc_to_dict(future_climate_stat_fc).getInfo()
            dcp_df = pd.DataFrame(dcp_dict)

            # add dates
            dcp_df = utils.add_date_info(dcp_df)

            # do some cleaning
            dcp_df['Precip-mm'] = dcp_df['Precip-rate'] * 86400 * 30
            dcp_df['Temp-mean'] = dcp_df['Temp-mean'] - 273.15
            dcp_df['Model'] = 'NEX-GDDP'
            dcp_df = dcp_df.drop('Precip-rate', 1)

            ##############################      Past CLIMATE      #################################3
            
            terra_climate = (ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
                             .filter(ee.Filter.date('1980-01-01', '2021-01-01'))
                             .select(['tmmn', 'tmmx', 'pr']))

            reduce_era5 = utils.create_reduce_region_function(
                geometry=aoi, reducer=ee.Reducer.mean(), scale=5000, crs='EPSG:4326')

            era5_stat_fc = (ee.FeatureCollection(terra_climate.map(reduce_era5))
                             .filter(ee.Filter.notNull(terra_climate.first().bandNames())))

            # Server to Client
            era5_dict = utils.fc_to_dict(era5_stat_fc).getInfo()

            # Panda data frame
            era5_df = pd.DataFrame(era5_dict)

            # do some cleaning
            era5_df = utils.add_date_info(era5_df)
            era5_df['Model'] = 'TERRA_CLIMATE'
            era5_df['Temp-mean'] = (era5_df['tmmn'] + era5_df['tmmx'])/2*0.1
            era5_df = era5_df.rename(columns={'pr': 'Precip-mm'})
            era5_df = era5_df.drop(['tmmn', 'tmmx'], 1)

            ############################## Combine #########################################

            # combine past and future climate data
            climate_df = pd.concat([era5_df, dcp_df], sort=True)

            # chart 1
            base = alt.Chart(climate_df).encode(
                x='Year:O',
                color='Model')

            line = base.mark_line().encode(
                y=alt.Y('median(Precip-mm):Q', title='Precipitation (mm/month)'))

            band = base.mark_errorband(extent='iqr').encode(
                y=alt.Y('Precip-mm:Q', title='Precipitation (mm/month)'))

            a = (band + line).properties(width=600, height=300)

            a.display()

            # chart 2
            line2 = alt.Chart(climate_df).mark_line().encode(
                x='Year:O',
                y='median(Temp-mean):Q',
                color='Model')

            band2 = alt.Chart(climate_df).mark_errorband(extent='iqr').encode(
                x='Year:O',
                y=alt.Y('Temp-mean:Q', title='Temperature (°C)'), color='Model')

            b = (band2 + line2).properties(width=600, height=300)

            b.display()
            
        except Exception as e:
            print(e)
            print('An error occurred during computation.')
            
submit.on_click(submit_click_chart3)

 - calculate ndvi and clean the table
 - join with weather data and livestock
 - write prediction model
 - test accuracy

Note to myself: my creteria of bad vegetation indices status is if summer average ndvi is 2 standard deviation lower than the previous five years of average. So basically we give score for each soum and flag it if z score is => 1 for the entire soum. 

#### near real time updates
Second function is list of ten soums that are below the threshold observed. 

#### machine learning models 
where data is continually trained to predict the status of rangeland degradation in near-real time
right hand size will be 

**To Do List**

- 1. Compare MODIS 500m vs Sentinel 10m vs Landsat 9
- 2. Map Z-score (The Current Status)
- 3. Machine Learning Based Classification to 4 Levels (Modelling Risk of Rangeland Degradation)
- 4. Predictive Model (Regression model)
- 5. Welfare Study

Environmental sustainability and Earth science
Earth Analytics

**To Do List (Research)**
- 1. Downlaod aggregate weather and vegitation
- 2. Make movie about grazing range tiff files
- 3. Biomass appendix
- 5. SGR and WGR doesn't change